# Load from DN3 (original BENDR)

In [ ]:
import dn3_ext
import torch 

In [ ]:
conv_encoder = dn3_ext.ConvEncoderBENDR(in_features=128, encoder_h=512, enc_width=(3,2,2,2), enc_downsample=(3,2,2,2))
print(sum(p.numel() for p in conv_encoder.parameters() if p.requires_grad))
t1 = torch.rand(16, 128, 2000)
conv_encoder(t1).shape

In [ ]:
contextualizer = dn3_ext.BENDRContextualizer(in_features=128)
print(sum(p.numel() for p in contextualizer.parameters() if p.requires_grad))
t2 = torch.rand(16, 128, 21)
contextualizer(t2).shape

In [ ]:
model = dn3_ext.BendingCollegeWav2Vec(conv_encoder, contextualizer)

In [ ]:
logits, z, mask = model.forward(t1)
logits.shape, z.shape, mask.shape

In [ ]:
encoder_layer = torch.nn.TransformerEncoderLayer(d_model=512, nhead=8, batch_first=True)
src = torch.rand(32, 10, 512)
out = encoder_layer(src)
out.shape

In [ ]:
def _generate_negatives(self, z, num_negatives=20):
        """Generate negative samples to compare each sequence location against"""
        batch_size, feat, full_len = z.shape
        z_k = z.permute([0, 2, 1]).reshape(-1, feat)
        with torch.no_grad():
            # candidates = torch.arange(full_len).unsqueeze(-1).expand(-1, self.num_negatives).flatten()
            negative_inds = torch.randint(0, full_len-1, size=(batch_size, full_len * num_negatives))
            # From wav2vec 2.0 implementation, I don't understand
            # negative_inds[negative_inds >= candidates] += 1

            for i in range(1, batch_size):
                negative_inds[i] += i * full_len

        z_k = z_k[negative_inds.view(-1)].view(batch_size, full_len, num_negatives, feat)
        return z_k, negative_inds

In [ ]:
z = torch.rand(16, 512, 10)
z_k, negative_inds = _generate_negatives(None, z)
z_k.shape, negative_inds.shape

# Dataloader test

In [ ]:
import datasets
import torch 
import os
import numpy as np

In [ ]:
root_dir = '/itet-stor/wolflu/deepeye_itetnas04/data/single_stream/'
# get dirs 2 levels deep
dirs = [os.path.join(root_dir, o) for o in os.listdir(root_dir) 
                    if os.path.isdir(os.path.join(root_dir,o))]
dirs = [os.path.join(o, p) for o in dirs for p in os.listdir(o) 
                    if os.path.isdir(os.path.join(o,p))]    

nb_dirs = len(dirs)
print(f'Found {nb_dirs} dirs')

# randomly assign dirs to train, val, test
# numpy random seed 
np.random.seed(42)
nums = np.random.permutation(nb_dirs)
train_dirs = [dirs[i] for i in nums[:int(0.8*nb_dirs)]]
val_dirs = [dirs[i] for i in nums[int(0.8*nb_dirs):int(0.9*nb_dirs)]]
test_dirs = [dirs[i] for i in nums[int(0.9*nb_dirs):]]

val_dirs

In [ ]:
dataset = datasets.MultiTaskDataset(
    root_dir="/itet-stor/wolflu/deepeye_itetnas04/data/single_stream/movie/EP000"
)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [ ]:
len(dataset)

In [ ]:
import pandas as pd 

df = pd.read_csv("/itet-stor/wolflu/net_scratch/projects/EEGEyeNet_experimental/data/stream/zuco/synch_min/YDG/YDG_NR1_EEG/substream_1.csv")
df.head()

In [ ]:
import torch 
from datasets_whole import VirtualTensorDataset

In [ ]:
dataset = VirtualTensorDataset(
    root_dir="/itet-stor/wolflu/net_scratch/projects/EEGEyeNet_experimental/data/stream_debug/movie",
    window_len=2000
)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [ ]:
for i, batch in enumerate(dataloader):
    print(i, batch.shape)
    if i == 0:
        break

## Finetuning 

In [ ]:
path = '/itet-stor/wolflu/net_scratch/projects/BENDR/reports/logs/20230224-133833'

In [ ]:
import torch 
import os

In [ ]:
params_path = os.path.join(path, 'version_0', 'hparams.yaml')
# load yaml file as dict 
import yaml
with open(params_path, 'r') as f:
    params = yaml.unsafe_load(f)

In [3]:
from model import create_bendr

model = create_bendr(params)


AttributeError: module 'pandas' has no attribute 'core'